# U-Tube Fluid Dynamics Simulation

This notebook simulates the fluid dynamics of a U-tube. The user can interact with the simulation by adjusting the fluid density, damping coefficient, initial fluid heights, and other parameters. The simulation displays the U-tube, as well as plots of the fluid's displacement, velocity, and phase space.

In [ ]:
# Importing packages
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Button, HBox, VBox, Output, FloatText, interactive_output, IntText, Layout, IntProgress
from IPython.display import display
from scipy.integrate import odeint
import time

In [ ]:
# Initial matplotlib settings [Optional]
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['axes.grid'] = True

In [ ]:
# Constant physical parameters
g = 9.8       # Gravitational acceleration (m/s²)

In [ ]:
# Creating interactive widgets
rho_slider = FloatSlider(value=1000, min=800, max=1200, step=10, description='ρ [kg/m³]=') # Slider for density
b_slider = FloatSlider(value=50, min=0, max=200, step=1, description=' b [kg/s]=') # Slider for the damping coefficient
hl_slider = FloatSlider(value=3, min=2, max=9, step=0.1, description='hl [m]=') # Slider for the initial fluid height on the left side
hr_slider = FloatSlider(value=2, min=2, max=9, step=0.1, description='hr [m]=') # Slider for the initial fluid height on the right side
S_slider = FloatSlider(value=0.05, min=0.01, max=0.2, step=0.01, description='S [m²]=') # Slider for the cross-sectional area of the tube
tt_slider = IntSlider(value=10, min=1, max=50, step=1, description='Time [s]=') # Slider for the total simulation time
steps_slider = IntSlider(value=100, min=1, max=100, step=1, description='Steps=') # Slider for the number of simulation steps

y0 = hl_slider.value - (hl_slider.value + hr_slider.value) / 2 # Calculation of the initial position of the fluid
y0_text = FloatText(value=y0, description='y0 [m]=', disabled=True) # Text field to display the initial position
L_text = FloatText(value=2 + hl_slider.value + hr_slider.value, description='L [m]=', disabled=True) # Text field to display the total length of the fluid
m_text = FloatText(value=rho_slider.value * S_slider.value * L_text.value, description='m [kg]=', disabled=True) # Text field to display the total mass of the fluid
E_text = FloatText(value=0, description='E [J]=', disabled=True) # Text field to display the total energy of the system

time_text = FloatText(value=0, description='t [s]=', disabled=True) # Text field to display the current simulation time
step_text = IntText(value=0, description='Step=', disabled=True) # Text field to display the current simulation step
progress_bar = IntProgress(value=0, min=0, max=steps_slider.value, description='Progress:', bar_style='info') # Progress bar to show the simulation progress

# State variables
simulation_active = False # Variable to control if the simulation is active or not
output = Output() # Object to display the simulation output
t = 0 # Variable to store the current time
y_pos = 0 # Variable to store the current fluid position
y_vel = 0 # Variable to store the current fluid velocity
y_equi = 0 # Variable to store the equilibrium position of the fluid
start_button = Button(description="Start", button_style='success') # Button to start the simulation

In [ ]:
# Function to solve the ODE
def solve_ode(params):
    L = 2 + params['hl'] + params['hr']
    m = params['rho'] * params['S'] * L

    def model(y, t, g, L, b, m):
        y_pos, y_vel = y
        dydt = [y_vel, -2*g*y_pos/L - b*y_vel/m]
        return dydt

    solution = odeint(model, [params['y0'], 0], t, args=(g, L, params['b'], m))
    return solution[:, 0], solution[:, 1]

In [ ]:
# Function to update the plots
def update_values(step, rho, b, hl, hr, S):
  global y_equi, simulation_active, y_pos, y_vel, t

  total_steps = steps_slider.value
  t = np.linspace(0, tt_slider.value, total_steps)

  if not simulation_active:
    y0_text.value = abs( ((hl + hr) / 2) - hl )
    L_text.value = 2 + hl + hr
    m_text.value = rho * S * (2 + hl + hr)

    y_pos = np.zeros(steps_slider.value)
    y_vel = np.zeros(steps_slider.value)
    params = {
        'step': step,
        'rho': rho,
        'b': b,
        'hl': hl,
        'hr': hr,
        'S': S,
        'y0': y0_text.value
    }

    y_equi = (params['hl'] + params['hr'])/ 2
    current_hl = params['hl']
    current_hr = params['hr']

    m = m_text.value
    L = L_text.value
    y0 = y0_text.value

    T = 0.5*m*y_vel[0]**2;
    V = m*g*y0**2/L;
    E = T + V;

    E_text.value = E

    create_plots(current_hl, current_hr, y_equi, t, y_pos, y_vel, step)

def update_plot(step):
  global y_equi, t, y_pos, y_vel

  time_text.value = t[step]
  total_steps = steps_slider.value
  t = np.linspace(0, tt_slider.value, total_steps)
  params = {
    'step': step,
    'rho': rho_slider.value,
    'b': b_slider.value,
    'hl': hl_slider.value,
    'hr': hr_slider.value,
    'S': S_slider.value,
    'y0': y0_text.value
  }

  current_hl = params['hl'] + (y_pos[step] - params['y0'])
  current_hr = params['hr'] - (y_pos[step] - params['y0'])

  m = m_text.value
  L = L_text.value
  y0 = current_hl - y_equi

  T = 0.5*m*y_vel[step]**2;
  V = m*g*y0**2/L;
  E = T + V;

  E_text.value = E

  create_plots(current_hl, current_hr, y_equi, t, y_pos, y_vel, step)

def create_plots(current_hl, current_hr, y_equi, t, y_pos, y_vel, step):
  with output:
    output.clear_output(wait=True)

    # Create a figure with 4 subplots
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2)

    # 1. U-tube drawing
    ax1.set_xlim(-1, 5)
    ax1.set_ylim(-1, 10)
    ax1.set_aspect('equal')
    ax1.set_title('U-Tube')

    # Draw fluid columns
    ax1.bar(0.5, current_hl, width=1, color='blue', alpha=0.5)
    ax1.bar(3.5, current_hr, width=1, color='blue', alpha=0.5)
    ax1.bar(2, 1, width=2, bottom=0, color='blue', alpha=0.5)  # Base

    # Tube outline
    ax1.plot([0, 0], [0, 10], 'k-', linewidth=2)
    ax1.plot([1, 1], [1, 10], 'k-', linewidth=2)
    ax1.plot([3, 3], [1, 10], 'k-', linewidth=2)
    ax1.plot([4, 4], [0, 10], 'k-', linewidth=2)
    ax1.plot([0, 4], [0, 0], 'k-', linewidth=2)
    ax1.plot([1, 3], [1, 1], 'k-', linewidth=2)

    # Equilibrium line
    ax1.axhline(y=y_equi, color='red', linestyle='--', label='Equilibrium')
    # ax1.legend()

    # 2. Position vs. time plot
    ax2.set_title('Displacement vs. Time')
    ax2.plot(t[:step+1], y_pos[:step+1], 'b-')
    ax2.set_xlabel('Time (s)')
    ax2.set_ylabel('Displacement (m)')

    # 3. Velocity vs. time plot
    ax3.set_title('Velocity vs. Time')
    ax3.plot(t[:step+1], y_vel[:step+1], 'r-')
    ax3.set_xlabel('Time (s)')
    ax3.set_ylabel('Velocity (m/s)')

    # 4. Phase space
    ax4.set_title('Phase Space')
    ax4.plot(y_pos[:step+1], y_vel[:step+1], 'g-')
    ax4.set_xlabel('Displacement (m)')
    ax4.set_ylabel('Velocity (m/s)')

    plt.tight_layout()
    plt.show()

In [ ]:
def on_start(b):
  global simulation_active, y_pos, y_vel, t

  total_steps = steps_slider.value
  t = np.linspace(0, tt_slider.value, total_steps)
  time_text.value = 0
  step_text.value = 0

  y_pos = np.zeros(steps_slider.value)
  y_vel = np.zeros(steps_slider.value)

  params = {
      'step': 0,
      'rho': rho_slider.value,
      'b': b_slider.value,
      'hl': hl_slider.value,
      'hr': hr_slider.value,
      'S': S_slider.value,
      'y0': y0_text.value
  }

  y_pos, y_vel = solve_ode(params)

  # Automatic animation
  start_button.disabled = True
  simulation_active = True
  rho_slider.disabled = True
  b_slider.disabled = True
  hl_slider.disabled = True
  hr_slider.disabled = True
  S_slider.disabled = True
  tt_slider.disabled = True
  steps_slider.disabled = True
  for step in range(total_steps):
    update_plot(step)
    step_text.value = step + 1
    progress_bar.value = (step+1)/steps_slider.value * 100
    time.sleep(0.01)
  start_button.disabled = False
  simulation_active = False
  rho_slider.disabled = False
  b_slider.disabled = False
  hl_slider.disabled = False
  hr_slider.disabled = False
  S_slider.disabled = False
  tt_slider.disabled = False
  steps_slider.disabled = False


start_button.on_click(on_start)

# Organize widgets
controls = VBox([
  HBox([rho_slider, b_slider]),
  HBox([hl_slider, hr_slider, S_slider]),
  HBox([y0_text, L_text, m_text, E_text]),
  HBox([time_text, step_text]),
  HBox([tt_slider, steps_slider, start_button])
], layout=Layout(justify_content='center'))

# Link the sliders to the update_plot function
interactive_output(update_values, {'step': step_text, 'rho': rho_slider, 'b': b_slider, 'hl': hl_slider, 'hr': hr_slider, 'S': S_slider})

# Display the Interface
display(controls, output, progress_bar)

Output()

IntProgress(value=0, bar_style='info', description='Progress:')